# Database Demo

Sample functionality for creating tables, inserting data and running similarity search with OgbujiPT.

__Notes:__

Notebook requirements:
```
pip install jupyter pgvector asyncpg sentence_transformers
```

This notebook will attempt to access a database named `PGv` at `sofola:5432`, using the username `oori` and password `example`. If you have a different setup, you can change the connection string in the first cell.

### Initial setup and Imports

In [1]:
import uuid

from ogbujipt.embedding.pgvector import DocDB, MessageDB

from sentence_transformers       import SentenceTransformer

DB_NAME = 'PGv'
HOST = 'sofola'
PORT = 5432
USER = 'oori'
PASSWORD = 'example'

e_model = SentenceTransformer('all-MiniLM-L6-v2')  # Load the embedding model

# Document Embedding

In [2]:
pacer_copypasta = [  # Demo document
    'The FitnessGram™ Pacer Test is a multistage aerobic capacity test that progressively gets more difficult as it continues.', 
    'The 20 meter pacer test will begin in 30 seconds. Line up at the start.', 
    'The running speed starts slowly, but gets faster each minute after you hear this signal.', 
    '[beep] A single lap should be completed each time you hear this sound.', 
    '[ding] Remember to run in a straight line, and run as long as possible.', 
    'The second time you fail to complete a lap before the sound, your test is over.', 
    'The test will begin on the word start. On your mark, get ready, start.'
]

### Connecting to the database

In [3]:
pacerDB = await DocDB.from_conn_params(
    embedding_model=e_model, 
    table_name='pacer',
    user=USER,
    password=PASSWORD,
    db_name=DB_NAME,
    host=HOST,
    port=int(PORT)
)

### Create Tables

In [4]:
await pacerDB.drop_table()    # Drop the table if one is found
await pacerDB.create_table()  # Create a new table

### Inserting Document

In [5]:
for index, text in enumerate(pacer_copypasta):   # For each line in the copypasta
    await pacerDB.insert(                        # Insert the line into the table
        content=text,                            # The text to be embedded
        title='Pacer Copypasta',                 # Title metadata
        page_numbers=[index],                    # Page number metadata
        tags=['fitness', 'pacer', 'copypasta'],  # Tag metadata
    )

print(f'Inserted {len(pacer_copypasta)} document chunks into the table')

Inserted 7 document chunks into the table


## Similarity search Document

### Searching the document with a perfect match

In [6]:
search_string = '[beep] A single lap should be completed each time you hear this sound.'
print(f'Semantic Searching data using search string:\n"{search_string}"')

sim_search = await pacerDB.search(
    query_string=search_string,  # string to search by
    limit=3                      # Number of results returned
)

Semantic Searching data using search string:
"[beep] A single lap should be completed each time you hear this sound."


In [7]:
print(f'RETURNED TITLE:\n"{sim_search[0]["title"]}"')                            # Print the title of the first result
print(f'RETURNED CONTENT:\n"{sim_search[0]["content"]}"')                        # Print the content of the first result
print(f'RETURNED COSINE SIMILARITY:\n{sim_search[0]["cosine_similarity"]:.2f}')  # Print the cosine similarity of the first result

RETURNED TITLE:
"Pacer Copypasta"
RETURNED CONTENT:
"[beep] A single lap should be completed each time you hear this sound."
RETURNED COSINE SIMILARITY:
0.00


In [8]:
print(f'RAW RETURN:')
sim_search

RAW RETURN:


[<Record cosine_similarity=0.0 title='Pacer Copypasta' content='[beep] A single lap should be completed each time you hear this sound.' page_numbers=[3] tags=['fitness', 'pacer', 'copypasta']>,
 <Record cosine_similarity=0.31445924384770496 title='Pacer Copypasta' content='The second time you fail to complete a lap before the sound, your test is over.' page_numbers=[5] tags=['fitness', 'pacer', 'copypasta']>,
 <Record cosine_similarity=0.634082588486436 title='Pacer Copypasta' content='The running speed starts slowly, but gets faster each minute after you hear this signal.' page_numbers=[2] tags=['fitness', 'pacer', 'copypasta']>]

### Searching the document with a partial match

In [9]:
search_string = 'straight'
print(f'Semantic Searching data using search string:\n"{search_string}"')

sim_search = await pacerDB.search(
    query_string=search_string,  # string to search by
    limit=3                      # Number of results returned
)

Semantic Searching data using search string:
"straight"


In [10]:
print(f'RETURNED TITLE:\n"{sim_search[0]["title"]}"')                            # Print the title of the first result
print(f'RETURNED CONTENT:\n"{sim_search[0]["content"]}"')                        # Print the content of the first result
print(f'RETURNED COSINE SIMILARITY:\n{sim_search[0]["cosine_similarity"]:.2f}')  # Print the cosine similarity of the first result

RETURNED TITLE:
"Pacer Copypasta"
RETURNED CONTENT:
"[ding] Remember to run in a straight line, and run as long as possible."
RETURNED COSINE SIMILARITY:
0.72


In [11]:
print('RAW RETURN:')  
sim_search

RAW RETURN:


[<Record cosine_similarity=0.7157614573027005 title='Pacer Copypasta' content='[ding] Remember to run in a straight line, and run as long as possible.' page_numbers=[4] tags=['fitness', 'pacer', 'copypasta']>,
 <Record cosine_similarity=0.8959717930563745 title='Pacer Copypasta' content='The test will begin on the word start. On your mark, get ready, start.' page_numbers=[6] tags=['fitness', 'pacer', 'copypasta']>,
 <Record cosine_similarity=0.9200870391648666 title='Pacer Copypasta' content='The running speed starts slowly, but gets faster each minute after you hear this signal.' page_numbers=[2] tags=['fitness', 'pacer', 'copypasta']>]

### Searching the document utilizing filters

In [12]:
search_string = 'straight'
print(f'Semantic Searching data using search string:\n"{search_string}"')

sim_search = await pacerDB.search(
    query_string=search_string,     # string to search by
    query_title='Pacer Copypasta',  # title to search by
    query_page_numbers=[3, 4, 5],   # Page number metadata
    query_tags=['fitness'],         # Tag metadata
    conjunctive=False,              # Conjunctive search
)

Semantic Searching data using search string:
"straight"


In [13]:
print(f'RETURNED TITLE:\n"{sim_search[0]["title"]}"')                            # Print the title of the first result
print(f'RETURNED CONTENT:\n"{sim_search[0]["content"]}"')                        # Print the content of the first result
print(f'RETURNED COSINE SIMILARITY:\n{sim_search[0]["cosine_similarity"]:.2f}')  # Print the cosine similarity of the first result

RETURNED TITLE:
"Pacer Copypasta"
RETURNED CONTENT:
"[ding] Remember to run in a straight line, and run as long as possible."
RETURNED COSINE SIMILARITY:
0.72


In [14]:
print('RAW RETURN:')  
sim_search

RAW RETURN:


[<Record cosine_similarity=0.7157614573027005 title='Pacer Copypasta' content='[ding] Remember to run in a straight line, and run as long as possible.' page_numbers=[4] tags=['fitness', 'pacer', 'copypasta']>]

# Cleanup

In [15]:
await pacerDB.drop_table()

# Chatlog Embedding

In [16]:
abbott_and_costello = [  # Demo chatlog
    {'role': 'system', 'content': 'The user is considering becoming a ballplayer. The assistant wants to make sure they knows what they\'re getting into.'},
    {'role': 'assistant', 'content': 'Strange as it may seem, they give ball players nowadays very peculiar names.'},
    {'role': 'user', 'content': 'Funny names?'},
    {'role': 'assistant', 'content': 'Nicknames, nicknames. Now, on the St. Louis team we have Who\'s on first, What\'s on second, I Don\'t Know is on third--'},
    {'role': 'user', 'content': 'That\'s what I want to find out. I want you to tell me the names of the fellows on the St. Louis team.'},
    {'role': 'assistant', 'content': "I'm telling you. Who is on first. What's on second. I Don't Know's on third--"},
    {'role': 'user', 'content': "You know the fellows' names?"},
    {'role': 'assistant', 'content': 'Yes.'},
    {'role': 'user', 'content': "Well, then who's playing first?"},
    {'role': 'assistant', 'content': 'Yes.'},
    {'role': 'user', 'content': "I mean the fellow's name on first base."},
    {'role': 'assistant', 'content': 'Who.'},
    {'role': 'user', 'content': "The fellow playin' first base."},
    {'role': 'assistant', 'content': 'Who.'},
    {'role': 'user', 'content': "The guy on first base."},
    {'role': 'assistant', 'content': 'Who is on first.'},
    {'role': 'user', 'content': "Well, what are you askin' me for?"},
    {'role': 'assistant', 'content': "I'm not asking you--I'm telling you. Who is on first."},
    {'role': 'user', 'content': "I'm asking you--who's on first?"},
    {'role': 'assistant', 'content': 'That\'s the man\'s name.'},
    {'role': 'user', 'content': "That's who's name?"},
    {'role': 'assistant', 'content': 'Yes.'},
]

### Connecting to the database

In [17]:
baseballDB = await MessageDB.from_conn_params(
    embedding_model=e_model, 
    table_name='baseball',
    user=USER,
    password=PASSWORD,
    db_name=DB_NAME,
    host=HOST,
    port=int(PORT)
)

### Create Tables

In [18]:
await baseballDB.drop_table()    # Drop the table if one is found
await baseballDB.create_table()  # Create a new table

### Inserting Chatlog

In [19]:
history_key = uuid.uuid4()        # Generate a key for the chatlog
for line in abbott_and_costello:  # For each line of dialog in the script
    await baseballDB.insert(      # Insert the message into the table
        history_key=history_key,  # The key for the chatlog
        role=line['role'],
        content=line['content'],
        metadata={'genre': 'comedy', 'year': 1938}
    )
print(f'Inserted {len(abbott_and_costello)} lines of dialog into the table with history key "{history_key}".')

Inserted 22 lines of dialog into the table with history key "0a52c9d2-0636-46f5-8313-8951ad5da31a".


## Similarity search Chatlog

### Searching the chatlog with a perfect match

In [20]:
search_string = 'nickname'
print(f'Semantic Searching data using search string: "{search_string}"')

sim_search = await baseballDB.search(
    history_key=history_key,
    query_string=search_string,
    limit=3
)

Semantic Searching data using search string: "nickname"


In [21]:
print(f'RETURNED TIMESTAMP:\n{sim_search[0]["ts"]}')                              # Print the timestamp of the first result
print(f'RETURNED MESSAGE:\n{sim_search[0]["role"]}: {sim_search[0]["content"]}')  # Print the message of the first result
print(f'RETURNED COSINE SIMILARITY:\n{sim_search[0]["cosine_similarity"]:.2f}')   # Print the cosine similarity of the first result

RETURNED TIMESTAMP:
2023-11-28 18:31:36.558134+00:00
RETURNED MESSAGE:
assistant: Nicknames, nicknames. Now, on the St. Louis team we have Who's on first, What's on second, I Don't Know is on third--
RETURNED COSINE SIMILARITY:
0.49


In [22]:
print('RAW RETURN:')
sim_search

RAW RETURN:


[{'ts': datetime.datetime(2023, 11, 28, 18, 31, 36, 558134, tzinfo=datetime.timezone.utc),
  'role': 'assistant',
  'content': "Nicknames, nicknames. Now, on the St. Louis team we have Who's on first, What's on second, I Don't Know is on third--",
  'metadata': {'genre': 'comedy', 'year': 1938},
  'cosine_similarity': 0.4910637432970839},
 {'ts': datetime.datetime(2023, 11, 28, 18, 31, 36, 537433, tzinfo=datetime.timezone.utc),
  'role': 'user',
  'content': 'Funny names?',
  'metadata': {'genre': 'comedy', 'year': 1938},
  'cosine_similarity': 0.561316881258301},
 {'ts': datetime.datetime(2023, 11, 28, 18, 31, 36, 823459, tzinfo=datetime.timezone.utc),
  'role': 'assistant',
  'content': "That's the man's name.",
  'metadata': {'genre': 'comedy', 'year': 1938},
  'cosine_similarity': 0.5729953094121991}]

### Searching the chatlog with a partial match

In [23]:
search_string = 'nickname'
print(f'Semantic Searching data using search string: "{search_string}"')

sim_search = await baseballDB.search(
    history_key=history_key,
    query_string=search_string,
    limit=3
)

Semantic Searching data using search string: "nickname"


In [24]:
print(f'RETURNED TIMESTAMP:\n{sim_search[0]["ts"]}\n')                              # Print the timestamp of the first result
print(f'RETURNED MESSAGE:\n{sim_search[0]["role"]}: {sim_search[0]["content"]}\n')  # Print the message of the first result
print(f'RETURNED COSINE SIMILARITY:\n{sim_search[0]["cosine_similarity"]:.2f}\n')   # Print the cosine similarity of the first result

RETURNED TIMESTAMP:
2023-11-28 18:31:36.558134+00:00

RETURNED MESSAGE:
assistant: Nicknames, nicknames. Now, on the St. Louis team we have Who's on first, What's on second, I Don't Know is on third--

RETURNED COSINE SIMILARITY:
0.49



In [25]:
print('RAW RETURN:')
sim_search

RAW RETURN:


[{'ts': datetime.datetime(2023, 11, 28, 18, 31, 36, 558134, tzinfo=datetime.timezone.utc),
  'role': 'assistant',
  'content': "Nicknames, nicknames. Now, on the St. Louis team we have Who's on first, What's on second, I Don't Know is on third--",
  'metadata': {'genre': 'comedy', 'year': 1938},
  'cosine_similarity': 0.4910637432970839},
 {'ts': datetime.datetime(2023, 11, 28, 18, 31, 36, 537433, tzinfo=datetime.timezone.utc),
  'role': 'user',
  'content': 'Funny names?',
  'metadata': {'genre': 'comedy', 'year': 1938},
  'cosine_similarity': 0.561316881258301},
 {'ts': datetime.datetime(2023, 11, 28, 18, 31, 36, 823459, tzinfo=datetime.timezone.utc),
  'role': 'assistant',
  'content': "That's the man's name.",
  'metadata': {'genre': 'comedy', 'year': 1938},
  'cosine_similarity': 0.5729953094121991}]

### Retrieving the entire chatlog

In [26]:
print(f'Retreiving chatlog "{history_key}" from database')
script_from_PG = await baseballDB.get_table(history_key=history_key)

Retreiving chatlog "0a52c9d2-0636-46f5-8313-8951ad5da31a" from database


In [27]:
print('RETURNED CHATLOG:')
for message in script_from_PG:
    print(f'{message["role"]}: {message["content"]}')

RETURNED CHATLOG:
system: The user is considering becoming a ballplayer. The assistant wants to make sure they knows what they're getting into.
assistant: Strange as it may seem, they give ball players nowadays very peculiar names.
user: Funny names?
assistant: Nicknames, nicknames. Now, on the St. Louis team we have Who's on first, What's on second, I Don't Know is on third--
user: That's what I want to find out. I want you to tell me the names of the fellows on the St. Louis team.
assistant: I'm telling you. Who is on first. What's on second. I Don't Know's on third--
user: You know the fellows' names?
assistant: Yes.
user: Well, then who's playing first?
assistant: Yes.
user: I mean the fellow's name on first base.
assistant: Who.
user: The fellow playin' first base.
assistant: Who.
user: The guy on first base.
assistant: Who is on first.
user: Well, what are you askin' me for?
assistant: I'm not asking you--I'm telling you. Who is on first.
user: I'm asking you--who's on first?
ass

In [28]:
print('RAW RETURN:')
script_from_PG

RAW RETURN:


[{'ts': datetime.datetime(2023, 11, 28, 18, 31, 36, 464581, tzinfo=datetime.timezone.utc),
  'role': 'system',
  'content': "The user is considering becoming a ballplayer. The assistant wants to make sure they knows what they're getting into.",
  'metadata': {'genre': 'comedy', 'year': 1938}},
 {'ts': datetime.datetime(2023, 11, 28, 18, 31, 36, 506643, tzinfo=datetime.timezone.utc),
  'role': 'assistant',
  'content': 'Strange as it may seem, they give ball players nowadays very peculiar names.',
  'metadata': {'genre': 'comedy', 'year': 1938}},
 {'ts': datetime.datetime(2023, 11, 28, 18, 31, 36, 537433, tzinfo=datetime.timezone.utc),
  'role': 'user',
  'content': 'Funny names?',
  'metadata': {'genre': 'comedy', 'year': 1938}},
 {'ts': datetime.datetime(2023, 11, 28, 18, 31, 36, 558134, tzinfo=datetime.timezone.utc),
  'role': 'assistant',
  'content': "Nicknames, nicknames. Now, on the St. Louis team we have Who's on first, What's on second, I Don't Know is on third--",
  'metadata'

### Cleanup

In [29]:
await baseballDB.drop_table()